In [ ]:
import sys
import pickle

import networkx as nx
import pickle
import itertools as it
import os
import shutil

import warnings
warnings.filterwarnings(action='once')

# if '/home/anna/BioSimSpace/python' not in sys.path:
#     sys.path.insert(1, '/home/anna/BioSimSpace/python')
# import BioSimSpace as BSS

# if '/home/anna/Documents/cinnabar' not in sys.path:
#     sys.path.insert(1, '/home/anna/Documents/cinnabar')
# import cinnabar

# print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *

# import BioSimSpace.Sandpit.Exscientia as BSS
# import BioSimSpace as BSS


BSS.__file__

In [ ]:
import BioSimSpace as BSS

def extract_ligand(system):

    ligand = None
    n_residues = [mol.nResidues() for mol in system]
    n_atoms = [mol.nAtoms() for mol in system]
    for i, (n_resi, n_at) in enumerate(zip(n_residues, n_atoms)):
        if n_resi == 1 and n_at > 5:
            ligand = system.getMolecule(i)
        else:
            pass
        if ligand:
            break
    
    return ligand

def merge_ligands( ligand_0, ligand_1):

    mapping = BSS.Align.matchAtoms(
                                ligand_0, ligand_1,
                                scoring_function="rmsd_align",
                                complete_rings_only=True
                                )           
    inv_mapping = {v: k for k, v in mapping.items()}

    ligand_1_a = BSS.Align.flexAlign(ligand_1, ligand_0, inv_mapping)

    merged_ligands = BSS.Align.merge(
        ligand_0, ligand_1_a, mapping,
        allow_ring_breaking=False,
        allow_ring_size_change=False
    )

    return merged_ligands


In [ ]:

pert = "lig_27~lig_59"

lig_0 = pert.split('~')[0]
lig_1 = pert.split('~')[1]

for name in ["sys"]:

    print(f"merging for {name}...")

    # Load equilibrated inputs for both ligands
    system0 = BSS.IO.readMolecules(
        [f"sample/{lig_0}_{name}_equil_solv.rst7", f"sample/{lig_0}_{name}_equil_solv.prm7"])
    system1 = BSS.IO.readMolecules(
        [f"sample/{lig_1}_{name}_equil_solv.rst7", f"sample/{lig_1}_{name}_equil_solv.prm7"])

    lig0 = extract_ligand(system0)
    lig1 = extract_ligand(system1)

    # BSS.IO.saveMolecules(f"lig_27_{name}", lig0, ["PRM7","RST7"])
    # BSS.IO.saveMolecules(f"lig_59_{name}", lig1, ["PRM7","RST7"])

    # ligs = merge_ligands(lig0, lig1)
    # system0.removeMolecules(lig0)
    # system_final = ligs + system0

    # BSS.IO.savePerturbableSystem(f"merged_{name}", system_final)


In [ ]:
for name in ["sys"]:

    print(f"merging for {name} (extracted ligand, reloaded)...")

    # Load equilibrated inputs for both ligands
    lig0r = BSS.IO.readMolecules(
        [f"sample/{lig_0}_{name}.rst7", f"sample/{lig_0}_{name}.prm7"])[0]
    lig1r = BSS.IO.readMolecules(
        [f"sample/{lig_1}_{name}.rst7", f"sample/{lig_1}_{name}.prm7"])[0]

    ligs = merge_ligands(lig0r, lig1r)

In [ ]:

def fep_prep(pert, prot_file, num_lambda_query, engine_query, main_dir, prep_dir):

    lig_1 = pert.split('~')[0]
    lig_2 = pert.split('~')[1]

    workdir = f"{main_dir}/outputs/{engine_query}/{lig_1}~{lig_2}" # pert dir

    # parse protocol file
    print("reading in the protocol file...")
    protocol = pipeline_protocol(prot_file) # instantiate the protocol as an object
    print("validating the protocol file...")
    protocol.validate() # validate all the input
    # print("rewriting the protocol file...")
    # protocol.rewrite_protocol() # rewrite protocol file
    # add the number of lambdas and engine to the protocol
    protocol.hmr_factor(3)
    protocol.num_lambda(num_lambda_query)
    protocol.engine(engine_query)
    if protocol.name():
        workdir += f"_{protocol.name()}"

    print("the protocol is now:")
    protocol.print_protocol()

    # instantiate each system as a fepprep class with the protocol
    fepprep_obj = fepprep(protocol=protocol)

    for name, leg in zip(["lig", "sys"], ["free", "bound"]):

        # Load equilibrated inputs for both ligands
        system_1 = BSS.IO.readMolecules(
            [f"{prep_dir}/{lig_1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_1}_{name}_equil_solv.prm7"])
        system_2 = BSS.IO.readMolecules(
            [f"{prep_dir}/{lig_2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_2}_{name}_equil_solv.prm7"])

        fepprep_obj.add_system(system_1, free_bound=leg, start_end="start")
        fepprep_obj.add_system(system_2, free_bound=leg, start_end="end")

    # remove any existing files in the workdir
    try:    
        remove_tree(workdir)
    except:
        pass
    # generate folder based on fepprep protocol (both or start)
    fepprep_obj.generate_folders(workdir)

In [ ]:
prot_file = "/home/anna/Documents/benchmark/mcl1_benchmark/execution_model/protocol_gromacs5.dat"
pert = "lig_33~lig_38"
num_lambda_query = 11
engine_query = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/mcl1_benchmark"
prep_dir = "/home/anna/Documents/benchmark/mcl1_benchmark/prep"

In [ ]:
fep_prep(pert, prot_file, num_lambda_query, engine_query, main_dir, prep_dir)


In [ ]:
transf = ["lig_60~lig_63", "lig_60~lig_61",
          "lig_35~lig_49","lig_27~lig_40",
          "lig_33~lig_38","lig_30~lig_40",
          "lig_27~lig_38","lig_56~lig_59",
          "lig_33~lig_59","lig_27~lig_59"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/mcl1_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["all_bonds"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
analysis_options.try_pickle(True)
analysis_options.print_protocol()

for method in methods:
    for trans in transf:

        # folder = path_to_dir = f"{main_dir}/outputs/{engine}/{method}_{trans}"
        # add_header_simfile(folder)
        # extraction = extract(folder)
        # # get the output from the folder to new folder
        # extraction.extract_output()
        # # get trajectory, will get rmsd by default
        # extraction.extract_frames(traj_lambdas=[], overwrite=True)

        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}_{trans}"
        # final_results_folder = f"{main_dir}/outputs_extracted/results"

        # # try:
        # # using the pipeline module for analysis
        # analysed_pert = analyse(path_to_dir)
        # analysed_pert.set_options(analysis_options)
        # analysed_pert.set_options({"try pickle":False})
        # avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # print(avg, error, repeats_tuple_list)
        # # analysed_pert.plot_graphs()
        # write_analysis_file(analysed_pert, final_results_folder, method=method)
        # # except Exception as e:
        # #     print(e)
        # #     print(f"could not analyse {path_to_dir}")

        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results/{method}"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")

In [ ]:
transf = ["lig_60~lig_63", "lig_60~lig_61",
          "lig_35~lig_49","lig_27~lig_40",
          "lig_33~lig_38","lig_30~lig_40",
          "lig_27~lig_38","lig_56~lig_59",
          "lig_33~lig_59","lig_27~lig_59"]
engine = "SOMD"
main_dir = "/home/anna/Documents/benchmark/extracted/mcl1"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["hmr_prune_constrained_h-bonds"]

# options
ana_file = f"/home/anna/Documents/benchmark/mcl1_benchmark/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
analysis_options.try_pickle(True)
analysis_options.print_protocol()
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:

        # folder = path_to_dir = f"{main_dir}/outputs/{engine}/{method}_{trans}"
        # add_header_simfile(folder)
        # extraction = extract(folder)
        # # get the output from the folder to new folder
        # extraction.extract_output()
        # # get trajectory, will get rmsd by default
        # extraction.extract_frames(traj_lambdas=[], overwrite=True)

        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}_{trans}"
        # final_results_folder = f"{main_dir}/outputs_extracted/results"

        # # try:
        # # using the pipeline module for analysis
        # analysed_pert = analyse(path_to_dir)
        # analysed_pert.set_options(analysis_options)
        # analysed_pert.set_options({"try pickle":False})
        # avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # print(avg, error, repeats_tuple_list)
        # # analysed_pert.plot_graphs()
        # write_analysis_file(analysed_pert, final_results_folder, method=method)
        # # except Exception as e:
        # #     print(e)
        # #     print(f"could not analyse {path_to_dir}")

        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{trans}"
        print(path_to_dir)
        final_results_folder = f"/home/anna/Documents/benchmark/mcl1_benchmark/outputs_extracted/results/{method}"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")